In [1]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.__version__

'2.0.0'

In [2]:
tf.executing_eagerly()

True

In [3]:
x = [[2.]]
m = tf.matmul(x, x)
# 矩阵a乘以矩阵b，生成a * b(矩阵相乘)
print("x matmul x = {}".format(m))

x matmul x = [[4.]]


In [4]:
a = tf.constant([[1, 2], [3, 4]])
# tf.constant创建常量
print(a)

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)


In [5]:
b = tf.add(a, 1)
# a+1
print(b)

tf.Tensor(
[[2 3]
 [4 5]], shape=(2, 2), dtype=int32)


In [6]:
print(a*b)
# 矩阵各元素对应相乘

tf.Tensor(
[[ 2  6]
 [12 20]], shape=(2, 2), dtype=int32)


In [7]:
print(tf.matmul(a, b))

tf.Tensor(
[[10 13]
 [22 29]], shape=(2, 2), dtype=int32)


In [8]:
import numpy as np

In [9]:
c = np.multiply(a, b)
# np.multiply(a, b)对应元素位置相乘
print(c)

[[ 2  6]
 [12 20]]


In [10]:
a.numpy()
# 将Tensor a转变为nump 数组

array([[1, 2],
       [3, 4]])

In [11]:
w = tf.Variable([[1.0]])
# tf.Variable创建变量
with tf.GradientTape() as tape:
# tf.GradientTape()计算梯度
    loss = w * w
grad = tape.gradient(loss, w)
print(grad)

tf.Tensor([[2.]], shape=(1, 1), dtype=float32)


In [12]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.mnist.load_data()
# tf.keras.datasets 加载keras内置数据集
x_train = x_train[:10000,:,:]
y_train = y_train[:10000]
x_test = x_test[:1000,:,:]
y_test = y_test[:1000]

In [13]:
x_train = tf.cast(x_train[..., tf.newaxis]/255, tf.float32)
# tf.cast 将x_train数据类型转变为float32
# tf.newaxis增加纬度
x_test = tf.cast(x_test[..., tf.newaxis]/255, tf.float32)
y_train = tf.keras.utils.to_categorical(y_train,10)
y_test = tf.keras.utils.to_categorical(y_test,10)
# tf.keras.utils.to_categorical 把类别标签转换为onehot编码

In [14]:
# 使用Sequential构建模型
mnist_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32,[3,3],activation='relu',
                          input_shape=(28,28,1)),
    tf.keras.layers.Conv2D(64,[3,3],activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10,activation="softmax")
])

In [15]:
mnist_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               1179776   
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0

In [16]:
# 用Model构建模型
inputs = tf.keras.Input(shape=(None,None,1),name="digits")
conv_1 = tf.keras.layers.Conv2D(16,[3,3],activation="relu")(inputs)
conv_2 = tf.keras.layers.Conv2D(16,[3,3],activation="relu")(conv_1)
ave_pool = tf.keras.layers.GlobalAveragePooling2D()(conv_2)
outputs = tf.keras.layers.Dense(10)(ave_pool)
mnist_model_2 = tf.keras.Model(inputs=inputs,outputs=outputs)

In [17]:
mnist_model_2.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
digits (InputLayer)          [(None, None, None, 1)]   0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 16)    160       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, None, None, 16)    2320      
_________________________________________________________________
global_average_pooling2d (Gl (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                170       
Total params: 2,650
Trainable params: 2,650
Non-trainable params: 0
_________________________________________________________________


In [18]:
mnist_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),
                    validation_split=0.1,shuffle=True,
                   loss = tf.keras.losses.categorical_crossentropy,
                   metrics = ["accuracy"])
# 损失函数SGD优化 

In [19]:
mnist_model_2.compile(loss = tf.keras.losses.categorical_crossentropy,
                      optimizer=tf.keras.optimizers.Adam(),
                     metrics=["accuracy"])
# 损失函数Adam优化

In [20]:
mnist_model.fit(x_train,y_train,batch_size=128,epochs=3)
# 模型拟合

Train on 10000 samples
Epoch 1/3
10000/10000 [==============================] - 4s 377us/sample - loss: 2.2993 - accuracy: 0.1117
Epoch 2/3
10000/10000 [==============================] - 2s 183us/sample - loss: 2.2738 - accuracy: 0.1520
Epoch 3/3
10000/10000 [==============================] - 2s 183us/sample - loss: 2.2481 - accuracy: 0.1938


In [21]:
mnist_model.evaluate(x_test,y_test)
# 模型评估，不输出预测结果

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[2.2252184562683106, 0.305]

In [22]:
mnist_model.predict(x_test)
# 模型预测


array([[0.10151026, 0.09252666, 0.10043221, ..., 0.1069214 , 0.100106  ,
        0.1020958 ],
       [0.11370236, 0.08542649, 0.10832264, ..., 0.08563392, 0.10040347,
        0.09040982],
       [0.09842034, 0.10683969, 0.10220093, ..., 0.09249648, 0.10642159,
        0.09792414],
       ...,
       [0.1508169 , 0.07628656, 0.09295099, ..., 0.07877033, 0.10629711,
        0.09116384],
       [0.11881833, 0.08513493, 0.0965813 , ..., 0.08462372, 0.10391635,
        0.09633109],
       [0.09952228, 0.08676473, 0.10197175, ..., 0.10234277, 0.10491342,
        0.10097966]], dtype=float32)

In [23]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train[:10000,:,:]
y_train = y_train[:10000]
x_test = x_test[:1000,:,:]
y_test = y_test[:1000]

In [24]:
dataset = tf.data.Dataset.from_tensor_slices(
(tf.cast(x_train[...,tf.newaxis]/255, tf.float32),
 tf.cast(y_train,tf.int64)))
# /255 将数据归一化
# tf.data.Dataset.from_tensor_slices 进行加载数据集
dataset = dataset.shuffle(1000).batch(32)
#  dataset.shuffle 随机打乱（1000）
# batch(32) 每32个打包为一个batch

In [25]:
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
# 学习率为1e-3的SGD优化
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss_history = []

In [26]:
for epoch in range(5):
    for (batch, (images,labels)) in enumerate (dataset):
        
        with tf.GradientTape() as tape:
            
            logits = mnist_model(images,training=True)
            loss_value = loss(labels,logits)
            
        grads = tape.gradient(loss_value,mnist_model.trainable_variables)
        # trainable_variables仅可以查看可训练的变量
        optimizer.apply_gradients(zip(grads,mnist_model.trainable_variables))
        
    print("Epoch {} finishted".format(epoch))

Epoch 0 finishted
Epoch 1 finishted
Epoch 2 finishted
Epoch 3 finishted
Epoch 4 finishted
